<a href="https://colab.research.google.com/github/yulmu99/Playdata/blob/main/Deep_learning/5_Learn_about_the_optimization_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing the model parameters

In [ ]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [ ]:
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download =True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:01<00:00, 15096004.17it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 272110.16it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5010492.34it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 28636972.14it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size = 64 )
test_dataloader =  DataLoader(test_data, batch_size =64)

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
        nn.ReLU()
       )
    
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

In [ ]:
model = NeuralNetwork()

##1.Setting hyperparameters
- Epoch
- batch_size
- learning_rate

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

##2.Add an optimization loop
- The Train Loop
> iterate over the training dataset and try to converge to optimal parameters.   
- The Validation/test Loop
> iterate over the test dataset to check if model performance is improving.

### Add a loss function
- `nn.MSELoss` (Mean Square Error) used for regression tasks
- `nn.NLLLoss` (Negative Log Likelihood) used for classification
- `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`

In [ ]:
# initialize the loss fuction
loss_fn = nn.CrossEntropyLoss()

### Optimization pass

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

### Backpropagation
- optimizer.zero_grad()
> 역전파 단계를 실행하기 전에 각 파라미터들의 변화도(gradient)를 0으로 재설정

- loss.backward()
> 역전파 단계: 모델의 학습을 가능한 모든 매개변수에 대해 손실의 변화도 계산

- optimizer.step()
> 변화도를 계산한 뒤에 `optimizer.step()`을 호춯하여 역전파 단계에서 수집된 변화도로 매개변수 조정

##3.Full implementation

In [ ]:
def train_loop(dataloader,model,loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch ,(X,y) in enumerate(dataloader):   #train data에서 64개씩 나눠서 학습
    #compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    #backpropagation 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item() , batch * len(X) 
      print(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')
    

In [ ]:
def test_loop(dataloader, model, loss_fn):    #test는 optimizer  X
  size = len(dataloader.dataset)
  test_loss, correct = 0,0

  with torch.no_grad():
    for X,y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()    #batch별 loss
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  
  test_loss /= size   #오답율
  correct /= size     #정답율
  print(f'Test Error" \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n')

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr =learning_rate)

epoch = 10
for t in range(epoch):
  print(f'Epoch {t+1}\n-------------------')
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------
loss: 2.296743  [    0/60000]
loss: 2.287503  [ 6400/60000]
loss: 2.279446  [12800/60000]
loss: 2.290223  [19200/60000]
loss: 2.257495  [25600/60000]
loss: 2.249572  [32000/60000]
loss: 2.243126  [38400/60000]
loss: 2.225015  [44800/60000]
loss: 2.226043  [51200/60000]
loss: 2.223431  [57600/60000]
Test Error" 
 Accuracy: 37.2%, Avg loss: 0.034851

Epoch 2
-------------------
loss: 2.188293  [    0/60000]
loss: 2.196789  [ 6400/60000]
loss: 2.185410  [12800/60000]
loss: 2.239280  [19200/60000]
loss: 2.144704  [25600/60000]
loss: 2.135642  [32000/60000]
loss: 2.132953  [38400/60000]
loss: 2.091267  [44800/60000]
loss: 2.113239  [51200/60000]
loss: 2.115427  [57600/60000]
Test Error" 
 Accuracy: 38.4%, Avg loss: 0.033107

Epoch 3
-------------------
loss: 2.042401  [    0/60000]
loss: 2.065198  [ 6400/60000]
loss: 2.043508  [12800/60000]
loss: 2.159828  [19200/60000]
loss: 1.979615  [25600/60000]
loss: 1.975802  [32000/60000]
loss: 1.976966  [38400/60000]
loss

## 4.Saving Models

In [ ]:
torch.save(model.state_dict(),'data/model.pth' )  #모델 저장

print('Saved PyTorch Model State to model.pth')

Saved PyTorch Model State to model.pth


##5.Loading Models

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load('data/model.pth')) # 모델 불러오기

<All keys matched successfully>

###  Prediction

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()  #  모델을 예측 모드로 변경
X, y = test_data[0][0], test_data[0][1]
with torch.no_grad():  #모델 예측
  pred = model(X)
  predicted, actual  = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}"" Actual: "{actual}"')

Predicted: "Ankle boot"" Actual: "Ankle boot"
